# DCGAN and CGAN Image Generation

This notebook demonstrates the implementation and training of Deep Convolutional Generative Adversarial Networks (DCGAN) and Conditional GANs (CGAN) for image generation. The notebook is organized into several sections that cover data preparation, model architecture, training, and evaluation.

## Table of Contents
1. [Introduction](#Introduction)
2. [Setup and Imports](#Setup-and-Imports)
3. [Data Loading and Preprocessing](#Data-Loading-and-Preprocessing)
4. [Model Architecture](#Model-Architecture)
5. [Training the Models](#Training-the-Models)
6. [Evaluation and Visualization](#Evaluation-and-Visualization)
7. [Conclusion](#Conclusion)


## Introduction

In this notebook, we will explore the use of DCGAN and CGAN models for generating images. DCGAN is a popular GAN architecture that uses convolutional networks, while CGAN extends this by conditioning the generation process on additional information, such as class labels. We will train these models on a dataset of images and evaluate their performance.

The key objectives are:
- Implement and train DCGAN and CGAN models.
- Compare the effectiveness of conditioned vs. unconditioned image generation.
- Visualize and analyze the quality of generated images.


## Setup and Imports

In [ ]:
# Import necessary libraries for deep learning and image processing
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.tensorboard import SummaryWriter
import time

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define hyperparameters
LEARNING_RATE = 2e-4  # Learning rate for both Generator and Discriminator
BATCH_SIZE = 128  # Batch size for training
IMAGE_SIZE = 64  # Image size (64x64)
CHANNELS_IMG = 3  # Number of channels in the input image (3 for RGB)
NOISE_DIM = 100  # Dimension of the noise vector
NUM_EPOCHS = 5000  # Number of epochs to train
FEATURES_DISC = 64  # Base number of features in the Discriminator
FEATURES_GEN = 64  # Base number of features in the Generator
LOG_DIR_REAL = "logs/real"  # Directory for TensorBoard logs (real images)
LOG_DIR_FAKE = "logs/fake"  # Directory for TensorBoard logs (fake images)


## Data Loading and Preprocessing

In [ ]:
# Data loading and preprocessing function
def load_data(batch_size=128, image_size=64):
    """
    Loads and preprocesses the dataset for GAN training.
    
    Args:
    - batch_size (int): The number of samples per batch.
    - image_size (int): The size of the images after resizing.
    
    Returns:
    - DataLoader: The DataLoader object for the dataset.
    """
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
    ])
    
    dataset = datasets.ImageFolder(root="/content/drive/MyDrive/THESIS_GAN_TRAINING_FILES/141228_combined/", transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader

# Load the dataset
dataloader = load_data(batch_size=BATCH_SIZE, image_size=IMAGE_SIZE)


## Model Architecture

In [ ]:
# Define the Discriminator model
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        """
        Initializes the Discriminator model.
        
        Args:
        - channels_img (int): Number of image channels (e.g., 3 for RGB).
        - features_d (int): Base number of features in the Discriminator.
        """
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid()
        )
    
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        """Returns a block of Conv2d -> BatchNorm2d -> LeakyReLU."""
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, x):
        return self.disc(x)

# Define the Generator model
class Generator(nn.Module):
    def __init__(self, noise_dim, channels_img, features_g):
        """
        Initializes the Generator model.
        
        Args:
        - noise_dim (int): Dimension of the input noise vector.
        - channels_img (int): Number of image channels (e.g., 3 for RGB).
        - features_g (int): Base number of features in the Generator.
        """
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            self._block(noise_dim, features_g * 16, 4, 1, 0),
            self._block(features_g * 16, features_g * 8, 4, 2, 1),
            self._block(features_g * 8, features_g * 4, 4, 2, 1),
            self._block(features_g * 4, features_g * 2, 4, 2, 1),
            nn.ConvTranspose2d(features_g * 2, channels_img, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )
    
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        """Returns a block of ConvTranspose2d -> BatchNorm2d -> ReLU."""
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
    
    def forward(self, x):
        return self.net(x)

    

In [ ]:
# Initialize the weights of the models
def initialize_weights(model):
    """
    Initializes the weights of the model based on the DCGAN paper.
    
    Args:
    - model (nn.Module): The model to initialize weights for.
    """
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

# Test the Generator and Discriminator models
def test_models():
    """Tests the Generator and Discriminator architectures."""
    N, in_channels, H, W = 8, 3, 64, 64
    noise_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
    
    print("All tests passed!")

# Run the test
test_models()

## Training the Models

In [ ]:
# Initialize models
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

# Print the model architectures
print("Generator Architecture:\n", gen)
print("\nDiscriminator Architecture:\n", disc)


In [ ]:
# Initialize models
gen = Generator(NOISE_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)
initialize_weights(gen)
initialize_weights(disc)

# Print the model architectures
print("Generator Architecture:\n", gen)
print("\nDiscriminator Architecture:\n", disc)


In [ ]:
# TensorBoard setup
fixed_noise = torch.randn(32, NOISE_DIM, 1, 1).to(device)
writer_real = SummaryWriter(LOG_DIR_REAL)
writer_fake = SummaryWriter(LOG_DIR_FAKE)
step = 0

# Training loop
gen.train()
disc.train()

begin = time.time()

for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        cur_batch_size = real.size(0)
        
        # Generate fake images
        noise = torch.randn(cur_batch_size, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)
        
        # Train Discriminator
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()
        
        # Train Generator
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()
        
        # Logging and visualization
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}")
            
            with torch.no_grad():
                fake = gen(fixed_noise)
                img_grid_real = torchvision.utils.make_grid(real[:32], normalize=True)
                img_grid_fake = torchvision.utils.make_grid(fake[:32], normalize=True)
                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
            
            step += 1

end = time.time()
print(f"Training Time: {(end - begin)/60:.2f} minutes")


## Evaluation and Visualization

In [2]:
# Launch TensorBoard to visualize training logs
# Uncomment and run the following lines in a separate cell if using Jupyter Notebook
# %reload_ext tensorboard
# %tensorboard --logdir=./logs


## Conclusion

In this notebook, we implemented and trained both DCGAN and CGAN models for image generation. The models were evaluated based on the quality of the generated images, demonstrating the effectiveness of GANs in producing realistic images. Further exploration could involve tuning hyperparameters, experimenting with different GAN architectures, or applying these models to more complex datasets.
